# APS1 - Redes Sociais

### Bruno Freitas do Nascimento Rodrigues e
### Nicolas Byung Kwan Cho

## Introdução



O dataset escolhido representa rede confiança entre usuários de uma plataforma de troca de Bitcoins, nesse caso a rede confiança tem como função marcar a reputação das carteiras para prevenir os usuários de fraudes e outros males. Contudo, no caso desta APS serão analisadas apenas as relações de trocas entre as carteiras sem se ater à confiabilidade delas.

## Importando bibliotecas e criando grafo

In [ ]:
import csv
import netpixi
from netpixi.integration.gt import Graph
import seaborn
from netpixi.integration.gt import *
from regression.integration.gt import *

Como a troca entre duas carteiras representa que uma está unicamente enviando moedas enquanto está apenas recebendo, o grafo será dirigido

In [ ]:
g = Graph(directed=True) # dirigido
g.add_ep('rating')
g.add_vp('dg')

In [ ]:
node_list = []
with open("soc-sign-bitcoinotc.csv", 'r', encoding='utf-8') as file:
    csv_file = csv.reader(file)
    for line in csv_file:
        a = int(line[0])
        b = int(line[1])
        c = int(line[2])
        if a not in node_list:
            node_list.append(a)
            g.add_vertex(a)
        if b not in node_list:
            node_list.append(b)
            g.add_vertex(b)
        g.add_edge(a, b)
        g.get_edge(a, b)['rating'] = c
        

## Análise exploratória

In [ ]:
print('número de vértices de g:', g.num_vertices())

In [ ]:
print('número de arestas de g:', g.num_edges())

In [ ]:
n = g.num_vertices()
m = g.num_edges()
if g.is_directed():
    max_edges = n * (n - 1)
else:
    max_edges = n * (n - 1) // 2
print('max de arestas possiveis:', max_edges)

In [ ]:
d = m / max_edges
print('densidade:', d)

É perceptível pela densidade que menos de 1% das transações possíveis estão sendo realizadas 

In [ ]:
in_degrees = []
for v in g.all_vertices():
    in_degrees.append(v.in_degree())

out_degrees = []
for v in g.all_vertices():
    out_degrees.append(v.out_degree())

degrees = []
for v in g.all_vertices():
    v['dg'] = v.total_degree()
    degrees.append(v.total_degree())

In [ ]:
data = gt_data(g)
data

Abaixo são adicionadas métricas para ser possível analisar as arestas entram e às saem dos vértices:

In [ ]:
data['in_degree'] = in_degrees
data['out_degree'] = out_degrees
data['degree'] = degrees
data

**Observação:**
Para os histogramas abaixo o range deles foi definido em 30, pois o comportamento de maior interesse está no intervalo inicial

In [ ]:
data['in_degree'].hist(bins=range(30))
data['in_degree'].describe()

In [ ]:
data['out_degree'].hist(bins=range(30))
data['out_degree'].describe()

In [ ]:
data['degree'].hist(bins=range(30))
data['degree'].describe()

Observando os valores de média e máximo dos histogramas acima, fica evidente que nessa rede de trocas há uma quantidade reduzida de carteiras que realizam muitas recebimentos e envios, enquanto a maioria dos usuários realizam poucas transações

## Análise visual

### SFDP - Scalable Force Directed Placement

In [ ]:
m = gt_draw.sfdp_layout(g)
gt_move(g, m)
gt_save(g, 'btc_sfdp.net.gz')

In [ ]:
r = netpixi.render('btc_sfdp.net.gz', infinite=True,aspect=1);

Aplicando ferramentas de visualização para a melhor identificação dos dados

In [ ]:
r.vertex_set_key('id')# adcionana labels para os vértices

In [ ]:
r.vertex_scale('dg', 0.1, 5)#Normalisa o tamanho dos vértices com base no degree

In [ ]:
r.vertex_default(color=0x00ff00, bwidth=3)#Configura cor e espessura da borda dos vértices
r.edge_default(alpha=0.1, width = 1)#Configura trasparência e espessura da borda das arestas

In [ ]:
r.graph(kscale=20)#Ajusta escala da chave em relação ao tamanho do vértice

## Referências

Dataset: https://snap.stanford.edu/data/soc-sign-bitcoin-otc.html